In [2]:
import sys

sys.path.append("../")

import pandas as pd
from dotenv import load_dotenv

load_dotenv()
import os
import sqlite3

DB_PATH = os.getenv("DB_PATH")

conn = sqlite3.connect(DB_PATH)

In [56]:
only_country = False

# Individuals Regions
df_ind_regions = pd.read_sql_query("SELECT * FROM individuals_regions", conn)

# Years
df_ind = pd.read_sql_query("SELECT * FROM individuals_main_information", conn)
df_ind_year = df_ind[["individual_wikidata_id", "birthyear"]].drop_duplicates()
df_ind_year = df_ind_year.dropna()
df_ind_year = df_ind_year[df_ind_year['birthyear']<=1850]


temporal_resolution = 10
df_ind_year["decade"] = df_ind_year["birthyear"].apply(lambda x: round(x / temporal_resolution) * temporal_resolution)

df_catalogs_id = pd.read_sql_query("SELECT * FROM individual_identifiers", conn)
df_catalogs = pd.read_sql_query("SELECT * FROM identifiers", conn)

if only_country:
    df_catalogs = df_catalogs[['identifiers_wikidata_id', 'country_name']].dropna()
    df_cat = pd.merge(df_catalogs_id, df_catalogs, on = 'identifiers_wikidata_id')
    df_cat = df_cat[['individual_wikidata_id','individual_name', 'country_name']].drop_duplicates()
    df_cat = df_cat.groupby(['individual_name', 'individual_wikidata_id'])['country_name'].count().rename('score').reset_index()
    df_cat = df_cat.sort_values('score', ascending=False).reset_index(drop=True)
else:
    df_cat = df_catalogs_id.groupby(['individual_name', 'individual_wikidata_id'])['identifiers_wikidata_id'].count().rename('score').reset_index()
    df_cat = df_cat.sort_values('score', ascending=False).reset_index(drop=True)
    
df_final = pd.merge(df_ind_year, df_cat, on = 'individual_wikidata_id')
df_final = pd.merge(df_final, df_ind_regions, on = ['individual_wikidata_id', 'individual_name'])
df_final.to_csv('../data/df_individuals_score.csv')

In [59]:
df_final.sort_values('score', ascending=False).sample(3)

,individual_wikidata_id,birthyear,decade,individual_name,score,region_code,region_name
432388,Q4589013,1808.0,1810,Auguste-Joseph De Mersseman,6,re_western_europe,Western Europe
240476,Q43139930,1849.0,1850,James Thomas Wheeler,4,re_british_islands,British Islands
57442,Q18528720,1811.0,1810,Emily Stackhouse,12,re_northwestern_europe,Northwestern Europe
